# Battle of the Cities Notebook

Capstone notebook for week 5 of IBM's Applied Data Science Capstone online course on Coursera. 

## Create and clean original *pandas* dataframe

Create a dataframe that includes cities within a 35mi commute distance to IBM's headquarters in Armonk, NY.

In [1]:
# install and import necessary packages
!pip install beautifulsoup4 # install the BeautifulSoup library for web scraping
!pip install lxml # install lxml parser to break down the html page into parts
!pip install requests # install the requests library
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print('...Packages installed!')

...Packages installed!


In [2]:
# create a pandas dataframe from a csv_file
df = pd.read_csv('cities_near_IBM_HQ.csv', header=None)

# rename column headers
labels = ['ZIP', 'State', 'County', 'City', 'TimeZone', 'Distance(km)']
df.columns = labels
df

,ZIP,State,County,City,TimeZone,Distance(km)
0,10504,NY,Westchester,Armonk,5,2
1,10594,NY,Westchester,Thornwood,5,4
2,6831,CT,Fairfield,Greenwich,5,6
3,10595,NY,Westchester,Valhalla,5,6
4,10570,NY,Westchester,Pleasantville,5,6
...,...,...,...,...,...,...
781,7193,NJ,Essex,Newark,5,56
782,7195,NJ,Essex,Newark,5,56
783,7198,NJ,Essex,Newark,5,56
784,7199,NJ,Essex,Newark,5,56


In [3]:
# drop unnecessary columns
columns = ['ZIP', 'TimeZone', 'Distance(km)']
df.drop(columns, axis=1, inplace=True)

# re-arrange column order
df = df[['City', 'State', 'County']]
df

,City,State,County
0,Armonk,NY,Westchester
1,Thornwood,NY,Westchester
2,Greenwich,CT,Fairfield
3,Valhalla,NY,Westchester
4,Pleasantville,NY,Westchester
...,...,...,...
781,Newark,NJ,Essex
782,Newark,NJ,Essex
783,Newark,NJ,Essex
784,Newark,NJ,Essex


In [4]:
#### NOTE
# keeping County column for now to see if easier to identify same similar part of name cities from others
# to see if actually different cities, or both within the same spot and can remove

#### Remove duplicates and verify

In [5]:
# remove duplicate rows with the same city-state combination
df = df.drop_duplicates(subset=['City', 'State'])
df.shape

(398, 3)

Notice the total rows dropped from 786 to 398. This is because the dataset sometimes contained multiple ZIP Codes for the same state-city combo. For this project though, solely interested in the cities and states, not ZIP Codes.

In [6]:
# verify one of the cities with duplicate city-state combos still has a row (didn't have all appearances removed)
df[df['City'].str.contains("Newark")]

,City,State,County
733,Newark,NJ,Essex


In [7]:
# verify only 1 appearance of each city in the dataframe
vc = df['City'].value_counts()
vc

Fairfield             2
Ridgefield            2
Ridgewood             2
Monroe                2
Hawthorne             2
                     ..
Katonah               1
Roslyn                1
Park Ridge            1
Cold Spring Harbor    1
Elmsford              1
Name: City, Length: 392, dtype: int64

In [8]:
# determine which cities appear twice
vc[vc>1]

Fairfield     2
Ridgefield    2
Ridgewood     2
Monroe        2
Hawthorne     2
Harrison      2
Name: City, dtype: int64

#### Investigate why these cities appear twice

In [9]:
# examine rows with Fairfield as city 
df[df['City'].str.contains("Fairfield")]

,City,State,County
267,Fairfield,CT,Fairfield
520,New Fairfield,CT,Fairfield
758,Fairfield,NJ,Essex


In [10]:
# examine rows with Ridgefield as city 
df[df['City'].str.contains("Ridgefield")]

,City,State,County
139,Ridgefield,CT,Fairfield
282,Ridgefield Park,NJ,Bergen
316,Ridgefield,NJ,Bergen


In [11]:
# examine rows with Ridgewood as city 
df[df['City'].str.contains("Ridgewood")]

,City,State,County
235,Ridgewood,NJ,Bergen
304,Ridgewood,NY,Queens


In [12]:
# examine rows with Monroe as city 
df[df['City'].str.contains("Monroe")]

,City,State,County
512,Monroe,NY,Orange
572,Monroe,CT,Fairfield


In [13]:
# examine rows with Hawthorne as city 
df[df['City'].str.contains("Hawthorne")]

,City,State,County
7,Hawthorne,NY,Westchester
324,Hawthorne,NJ,Passaic


In [14]:
# examine rows with Harrison as city 
df[df['City'].str.contains("Harrison")]

,City,State,County
5,West Harrison,NY,Westchester
33,Harrison,NY,Westchester
749,Harrison,NJ,Hudson


The duplicate cites are actually different cities located in different states and counties. Hence, will keep these duplicate cities in the dataframe since they are indeed different locations, and not the same.

#### Finish cleaning dataframe

In [15]:
# sort the City, State column in descending order
df = df.sort_values(by='City', ascending=True)

# reset index
df = df.reset_index(drop=True) # drops the previous index column
df

,City,State,County
0,Albertson,NY,Nassau
1,Allendale,NJ,Bergen
2,Alpine,NJ,Bergen
3,Amawalk,NY,Westchester
4,Amityville,NY,Suffolk
...,...,...,...
393,Woodside,NY,Queens
394,Wyandanch,NY,Suffolk
395,Wyckoff,NJ,Bergen
396,Yonkers,NY,Westchester


---

## Create and clean 3 state dataframes

Create 3 state dataframes using data from tables on different Wikipedia pages that contain 2010 population data for towns in NY, CT, and NJ.

#### New York dataframe

In [16]:
# pull code from the web via the requests library
source = requests.get('https://en.wikipedia.org/wiki/List_of_towns_in_New_York_(state)').text

# use BeautifulSoup to scrape and lxml to parse
soup = BeautifulSoup(source, 'lxml')

In [17]:
# prettify method indents the html code, making it easier to read code within each tag
# commenting this line of code out because don't know how to hide the lengthy code blob on GitHub
# print(soup.prettify()) 

In [18]:
# retrieve only the text within the <table> tag, then read the data into a pandas dataframe
table = soup.find_all('table')[1] # [1] is an index value - retrieves only the second table from the webpage
df_ny = pd.read_html(str(table))[0] # [0] converts data from a list to a dataframe, remove it to see!
df_ny

,Town,County,Pop.(2010),Land(sq mi),Water(sq mi),Coordinates,GEO ID(FIPS code),ANSI code(GNIS ID)
0,Adams,Jefferson,5143,42.270,0.153,"43.844658, -76.054289",3604500210,978655
1,Addison,Steuben,2595,25.545,0.142,"42.132164, -77.234422",3610100287,978656
2,Afton,Chenango,2851,45.836,0.679,"42.226305, -75.526838",3601700353,978657
3,Alabama,Genesee,1869,42.370,0.406,"43.090053, -78.366480",3603700474,978658
4,Albion,Orleans,8468,25.257,0.093,"43.226343, -78.209387",3607301044,978660
...,...,...,...,...,...,...,...,...
928,Wright,Schoharie,1539,28.636,0.082,"42.673493, -74.207635",3609583195,979658
929,Yates,Orleans,2559,37.387,0.000,"43.339463, -78.388285",3607383448,979659
930,York,Livingston,3397,49.100,0.007,"42.866835, -77.888813",3605184022,979661
931,Yorkshire,Cattaraugus,3913,36.249,0.145,"42.479169, -78.520073",3600984055,979662


In [19]:
# rewrite df_ny with only desired columns
df_ny = df_ny[['Town', 'County', 'Pop.(2010)']]

# rename column headers
labels = ['City', 'County', 'Pop2010']
df_ny.columns = labels

# add Armonk, NY to df_ny (not included in Wikipedia's table)
df_ny = df_ny.append({'City':'Armonk', 'County':'Westchester', 'Pop2010':4330}, ignore_index=True)
df_ny.tail()

,City,County,Pop2010
929,Yates,Orleans,2559
930,York,Livingston,3397
931,Yorkshire,Cattaraugus,3913
932,Yorktown,Westchester,36081
933,Armonk,Westchester,4330


#### Connecticut dataframe

In [20]:
# pull code from the web via the requests library
source = requests.get('https://en.wikipedia.org/wiki/List_of_towns_in_Connecticut').text

# use BeautifulSoup to scrape and lxml to parse
soup = BeautifulSoup(source, 'lxml')

In [21]:
# prettify method indents the html code, making it easier to read code within each tag
# commenting this line of code out because don't know how to hide the lengthy code blob on GitHub
# print(soup.prettify()) 

In [22]:
# retrieve only the text within the <table> tag, then read the data into a pandas dataframe
table = soup.find_all('table')[1]
df_ct = pd.read_html(str(table))[0] # [0] converts data from a list to a dataframe, remove it to see!
df_ct

,Number,Town,Designation,Dateestablished,Land area(square miles),Population(in 2010),Form ofgovernment,County,Council of Governments,Native Americanname
0,1.0,Andover,Town,1848,15.46,3303,Town meeting,Tolland County,Capitol Region,NaN
1,2.0,Ansonia,City,1889,6.03,19249,Mayor-council,New Haven County,Naugatuck Valley,NaN
2,3.0,Ashford,Town,1714,38.79,4100,Town meeting,Windham County,Northeast CT,NaN
3,4.0,Avon,Town,1830,23.12,18098,Council-manager,Hartford County,Capitol Region,NaN
4,5.0,Barkhamsted,Town,1779,36.22,3620,Town meeting,Litchfield County,Northwest Hills,NaN
...,...,...,...,...,...,...,...,...,...,...
165,166.0,Wolcott,Town,1796,20.43,16680,Mayor-council,New Haven County,Naugatuck Valley,NaN
166,167.0,Woodbridge,Town,1784,18.83,8990,Town meeting,New Haven County,South Central,NaN
167,168.0,Woodbury,Town,1673,36.47,9975,Town meeting,Litchfield County,Naugatuck Valley,Pomperaug
168,169.0,Woodstock,Town,1686,60.54,7964,Town meeting,Windham County,Northeast CT,NaN


In [23]:
# rewrite df_ct with only desired columns
df_ct = df_ct[['Town', 'County', 'Population(in 2010)']]

# rename column headers
labels = ['City', 'County', 'Pop2010']
df_ct.columns = labels

# remove NaN row (was a Totals column on the webpage)
df_ct.dropna(axis=0, inplace=True)
df_ct

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,City,County,Pop2010
0,Andover,Tolland County,3303
1,Ansonia,New Haven County,19249
2,Ashford,Windham County,4100
3,Avon,Hartford County,18098
4,Barkhamsted,Litchfield County,3620
...,...,...,...
164,Windsor Locks,Hartford County,12498
165,Wolcott,New Haven County,16680
166,Woodbridge,New Haven County,8990
167,Woodbury,Litchfield County,9975


#### New Jersey dataframe

In [24]:
# pull code from the web via the requests library
source = requests.get('https://en.wikipedia.org/wiki/List_of_municipalities_in_New_Jersey').text

# use BeautifulSoup to scrape and lxml to parse
soup = BeautifulSoup(source, 'lxml')

In [25]:
# prettify method indents the html code, making it easier to read code within each tag
# commenting this line of code out because don't know how to hide the lengthy code blob on GitHub
# print(soup.prettify()) 

In [26]:
# retrieve only the text within the <table> tag, then read the data into a pandas dataframe
table = soup.find_all('table')[0] # use [0] because on this webpage the table appears first
df_nj = pd.read_html(str(table))[0] # [0] converts data from a list to a dataframe, remove it to see!
df_nj

,2010 Rank,Municipality,County,Population in 2010,Populationin 2017[3],Municipaltype,Form ofgovernment,Communityestablished,Incorporated[4]
0,1,Newark,Essex,277140,285154,City,Faulkner Act (mayor–council),1666,1693[note 1]
1,2,Jersey City,Hudson,247597,270753,City,Faulkner Act (mayor–council),1630,1838
2,3,Paterson,Passaic,146199,148678,City,Faulkner Act (mayor–council),1791,1831[note 2]
3,4,Elizabeth,Union,124969,130215,City,Faulkner Act (mayor–council),1664,1855
4,5,Edison,Middlesex,99967,102450,Township,Faulkner Act (mayor–council),1666[5],1870[note 3]
...,...,...,...,...,...,...,...,...,...
560,561,Loch Arbour,Monmouth,194,183,Village,Walsh Act,1957,1957
561,562,Teterboro,Bergen,67,69,Borough,1923 Municipal Manager Law,1917,1917
562,563,Walpack Township,Sussex,16,11,Township,NaN,1731,1798
563,564,Pine Valley,Camden,12,11,Borough,Walsh Act,NaN,1929


In [27]:
# rewrite df_nj with only desired columns
df_nj = df_nj[['Municipality', 'County', 'Population in 2010']]

# rename column headers
labels = ['City', 'County', 'Pop2010']
df_nj.columns = labels
df_nj

,City,County,Pop2010
0,Newark,Essex,277140
1,Jersey City,Hudson,247597
2,Paterson,Passaic,146199
3,Elizabeth,Union,124969
4,Edison,Middlesex,99967
...,...,...,...
560,Loch Arbour,Monmouth,194
561,Teterboro,Bergen,67
562,Walpack Township,Sussex,16
563,Pine Valley,Camden,12


---

## Merge state dataframes with original

Create 3 *new* dataframes by merging each state dataframe with the original dataframe; clean each so the newly merged dataframes only contain matching rows with a 2010 population of 20000 or greater and are from that state.  

Remove rows with NaN values because if no Pop2010 data is available then that city from csv_file's list is likely too small/ not actually a city or not from within the merged state. If no State/County_x data is available then the town from the Wikipedia table is likely not within the 35mi commute distance to IBM's headquarters because it was not a city from the csv_file.

#### New York merge and highpop dataframe

In [28]:
# perform an outer merge of the original df and df_ny on the City column
df_ny_merge = pd.merge(df, df_ny, on='City', how='outer')
df_ny_merge

,City,State,County_x,County_y,Pop2010
0,Albertson,NY,Nassau,NaN,NaN
1,Allendale,NJ,Bergen,NaN,NaN
2,Alpine,NJ,Bergen,NaN,NaN
3,Amawalk,NY,Westchester,NaN,NaN
4,Amityville,NY,Suffolk,NaN,NaN
...,...,...,...,...,...
1275,Wright,NaN,NaN,Schoharie,1539.0
1276,Yates,NaN,NaN,Orleans,2559.0
1277,York,NaN,NaN,Livingston,3397.0
1278,Yorkshire,NaN,NaN,Cattaraugus,3913.0


In [29]:
# drop rows that contain NaN
df_ny_merge.dropna(axis=0, inplace=True)
df_ny_merge.head(10)

,City,State,County_x,County_y,Pop2010
8,Armonk,NY,Westchester,Westchester,4330.0
10,Baldwin,NY,Nassau,Chemung,832.0
16,Bedford,NY,Westchester,Westchester,17335.0
23,Bethel,CT,Fairfield,Sullivan,4255.0
27,Blooming Grove,NY,Orange,Orange,18028.0
35,Bridgewater,CT,Litchfield,Oneida,1522.0
38,Brookfield,CT,Fairfield,Madison,2545.0
45,Carmel,NY,Putnam,Putnam,34305.0
53,Chester,NY,Orange,Orange,11981.0
54,Chester,NY,Orange,Warren,3355.0


In [30]:
# remove rows not of NY state data
df_ny_merge = df_ny_merge[df_ny_merge.State == 'NY']
df_ny_merge.head(10)

,City,State,County_x,County_y,Pop2010
8,Armonk,NY,Westchester,Westchester,4330.0
10,Baldwin,NY,Nassau,Chemung,832.0
16,Bedford,NY,Westchester,Westchester,17335.0
27,Blooming Grove,NY,Orange,Orange,18028.0
45,Carmel,NY,Putnam,Putnam,34305.0
53,Chester,NY,Orange,Orange,11981.0
54,Chester,NY,Orange,Warren,3355.0
64,Cornwall,NY,Orange,Orange,12646.0
88,Eastchester,NY,Westchester,Westchester,32363.0
104,Fishkill,NY,Dutchess,Dutchess,22107.0


In [31]:
# sort by Pop2010
df_ny_merge = df_ny_merge.sort_values(by='Pop2010', ascending=False)

# reset index
df_ny_merge = df_ny_merge.reset_index(drop=True) # drops the previous index column
df_ny_merge.head(10)

,City,State,County_x,County_y,Pop2010
0,Hempstead,NY,Nassau,Nassau,759757.0
1,Oyster Bay,NY,Nassau,Nassau,293214.0
2,Huntington,NY,Suffolk,Suffolk,203264.0
3,Smithtown,NY,Suffolk,Suffolk,117801.0
4,Rye,NY,Westchester,Westchester,45928.0
5,Monroe,NY,Orange,Orange,39912.0
6,Ossining,NY,Westchester,Westchester,37674.0
7,Haverstraw,NY,Rockland,Rockland,36634.0
8,Carmel,NY,Putnam,Putnam,34305.0
9,Eastchester,NY,Westchester,Westchester,32363.0


In [32]:
# create a new df of only rows with 20000 or greater population and the row with Armonk's data
df_ny_highpop = df_ny_merge.loc[(df_ny_merge['Pop2010'] >= 20000) | (df_ny_merge['City'] == 'Armonk')] # | = or ; & = and
df_ny_highpop

,City,State,County_x,County_y,Pop2010
0,Hempstead,NY,Nassau,Nassau,759757.0
1,Oyster Bay,NY,Nassau,Nassau,293214.0
2,Huntington,NY,Suffolk,Suffolk,203264.0
3,Smithtown,NY,Suffolk,Suffolk,117801.0
4,Rye,NY,Westchester,Westchester,45928.0
5,Monroe,NY,Orange,Orange,39912.0
6,Ossining,NY,Westchester,Westchester,37674.0
7,Haverstraw,NY,Rockland,Rockland,36634.0
8,Carmel,NY,Putnam,Putnam,34305.0
9,Eastchester,NY,Westchester,Westchester,32363.0


#### Connecticut merge and highpop dataframe

In [33]:
# perform an outer merge of the original df and df_ct on the City column
df_ct_merge = pd.merge(df, df_ct, on='City', how='outer')
df_ct_merge

,City,State,County_x,County_y,Pop2010
0,Albertson,NY,Nassau,NaN,NaN
1,Allendale,NJ,Bergen,NaN,NaN
2,Alpine,NJ,Bergen,NaN,NaN
3,Amawalk,NY,Westchester,NaN,NaN
4,Amityville,NY,Suffolk,NaN,NaN
...,...,...,...,...,...
532,Windsor,NaN,NaN,Hartford County,29044.0
533,Windsor Locks,NaN,NaN,Hartford County,12498.0
534,Wolcott,NaN,NaN,New Haven County,16680.0
535,Woodbridge,NaN,NaN,New Haven County,8990.0


In [34]:
# drop rows that contain NaN
df_ct_merge.dropna(axis=0, inplace=True)
df_ct_merge.head(10)

,City,State,County_x,County_y,Pop2010
23,Bethel,CT,Fairfield,Fairfield County,18584.0
26,Bloomfield,NJ,Essex,Hartford County,20486.0
34,Bridgeport,CT,Fairfield,Fairfield County,144229.0
35,Bridgewater,CT,Litchfield,Litchfield County,1727.0
38,Brookfield,CT,Fairfield,Fairfield County,16452.0
39,Brooklyn,NY,Kings,Windham County,8210.0
53,Chester,NY,Orange,Middlesex County,3994.0
63,Cornwall,NY,Orange,Litchfield County,1420.0
73,Danbury,CT,Fairfield,Fairfield County,80893.0
74,Darien,CT,Fairfield,Fairfield County,20732.0


In [35]:
# remove rows not of CT state data
df_ct_merge = df_ct_merge[df_ct_merge.State == 'CT']
df_ct_merge.head(10)

,City,State,County_x,County_y,Pop2010
23,Bethel,CT,Fairfield,Fairfield County,18584.0
34,Bridgeport,CT,Fairfield,Fairfield County,144229.0
35,Bridgewater,CT,Litchfield,Litchfield County,1727.0
38,Brookfield,CT,Fairfield,Fairfield County,16452.0
73,Danbury,CT,Fairfield,Fairfield County,80893.0
74,Darien,CT,Fairfield,Fairfield County,20732.0
88,Easton,CT,Fairfield,Fairfield County,7490.0
99,Fairfield,CT,Fairfield,Fairfield County,59404.0
131,Greenwich,CT,Fairfield,Fairfield County,61171.0
213,Monroe,CT,Fairfield,Fairfield County,19479.0


In [36]:
# sort by Pop2010
df_ct_merge = df_ct_merge.sort_values(by='Pop2010', ascending=False)
df_ct_merge

# reset index
df_ct_merge = df_ct_merge.reset_index(drop=True) # drops the previous index column
df_ct_merge.head(10)

,City,State,County_x,County_y,Pop2010
0,Bridgeport,CT,Fairfield,Fairfield County,144229.0
1,Stamford,CT,Fairfield,Fairfield County,122643.0
2,Norwalk,CT,Fairfield,Fairfield County,85603.0
3,Danbury,CT,Fairfield,Fairfield County,80893.0
4,Greenwich,CT,Fairfield,Fairfield County,61171.0
5,Fairfield,CT,Fairfield,Fairfield County,59404.0
6,Stratford,CT,Fairfield,Fairfield County,51384.0
7,Shelton,CT,Fairfield,Fairfield County,39559.0
8,Trumbull,CT,Fairfield,Fairfield County,36018.0
9,Newtown,CT,Fairfield,Fairfield County,27560.0


In [37]:
# create a new df of only rows with 20000 or greater population
df_ct_highpop = df_ct_merge.loc[df_ct_merge['Pop2010'] >= 20000]
df_ct_highpop

,City,State,County_x,County_y,Pop2010
0,Bridgeport,CT,Fairfield,Fairfield County,144229.0
1,Stamford,CT,Fairfield,Fairfield County,122643.0
2,Norwalk,CT,Fairfield,Fairfield County,85603.0
3,Danbury,CT,Fairfield,Fairfield County,80893.0
4,Greenwich,CT,Fairfield,Fairfield County,61171.0
5,Fairfield,CT,Fairfield,Fairfield County,59404.0
6,Stratford,CT,Fairfield,Fairfield County,51384.0
7,Shelton,CT,Fairfield,Fairfield County,39559.0
8,Trumbull,CT,Fairfield,Fairfield County,36018.0
9,Newtown,CT,Fairfield,Fairfield County,27560.0


#### New Jersey merge and highpop dataframe

In [38]:
# perform an outer merge of the original df and df_nj on the City column
df_nj_merge = pd.merge(df, df_nj, on='City', how='outer')
df_nj_merge

,City,State,County_x,County_y,Pop2010
0,Albertson,NY,Nassau,NaN,NaN
1,Allendale,NJ,Bergen,Bergen,6505.0
2,Alpine,NJ,Bergen,Bergen,1849.0
3,Amawalk,NY,Westchester,NaN,NaN
4,Amityville,NY,Suffolk,NaN,NaN
...,...,...,...,...,...
855,Cape May Point,NaN,NaN,Cape May,291.0
856,Loch Arbour,NaN,NaN,Monmouth,194.0
857,Walpack Township,NaN,NaN,Sussex,16.0
858,Pine Valley,NaN,NaN,Camden,12.0


In [39]:
# drop rows that contain NaN
df_nj_merge.dropna(axis=0, inplace=True)
df_nj_merge.head(10)

,City,State,County_x,County_y,Pop2010
1,Allendale,NJ,Bergen,Bergen,6505.0
2,Alpine,NJ,Bergen,Bergen,1849.0
19,Belleville,NJ,Essex,Essex,35926.0
22,Bergenfield,NJ,Bergen,Bergen,26652.0
26,Bloomfield,NJ,Essex,Essex,47315.0
28,Bloomingdale,NJ,Passaic,Passaic,7656.0
29,Bogota,NJ,Bergen,Bergen,8187.0
41,Caldwell,NJ,Essex,Essex,7822.0
44,Carlstadt,NJ,Bergen,Bergen,6127.0
47,Cedar Grove,NJ,Essex,Essex,12373.0


In [40]:
# remove rows not of NJ state data
df_nj_merge = df_nj_merge[df_nj_merge.State == 'NJ']
df_nj_merge.head(10)

,City,State,County_x,County_y,Pop2010
1,Allendale,NJ,Bergen,Bergen,6505.0
2,Alpine,NJ,Bergen,Bergen,1849.0
19,Belleville,NJ,Essex,Essex,35926.0
22,Bergenfield,NJ,Bergen,Bergen,26652.0
26,Bloomfield,NJ,Essex,Essex,47315.0
28,Bloomingdale,NJ,Passaic,Passaic,7656.0
29,Bogota,NJ,Bergen,Bergen,8187.0
41,Caldwell,NJ,Essex,Essex,7822.0
44,Carlstadt,NJ,Bergen,Bergen,6127.0
47,Cedar Grove,NJ,Essex,Essex,12373.0


In [41]:
# sort by Pop2010
df_nj_merge = df_nj_merge.sort_values(by='Pop2010', ascending=False)
df_nj_merge

# reset index
df_nj_merge = df_nj_merge.reset_index(drop=True) # drops the previous index column
df_nj_merge.head(10)

,City,State,County_x,County_y,Pop2010
0,Newark,NJ,Essex,Essex,277140.0
1,Jersey City,NJ,Hudson,Hudson,247597.0
2,Paterson,NJ,Passaic,Passaic,146199.0
3,Clifton,NJ,Passaic,Passaic,84136.0
4,Passaic,NJ,Passaic,Passaic,69781.0
5,Union City,NJ,Hudson,Hudson,66455.0
6,East Orange,NJ,Essex,Essex,64270.0
7,North Bergen,NJ,Hudson,Hudson,60773.0
8,Wayne,NJ,Passaic,Passaic,54717.0
9,Hoboken,NJ,Hudson,Hudson,50005.0


In [42]:
# create a new df of only rows with 20000 or greater population
df_nj_highpop = df_nj_merge.loc[df_nj_merge['Pop2010'] >= 20000]
df_nj_highpop

,City,State,County_x,County_y,Pop2010
0,Newark,NJ,Essex,Essex,277140.0
1,Jersey City,NJ,Hudson,Hudson,247597.0
2,Paterson,NJ,Passaic,Passaic,146199.0
3,Clifton,NJ,Passaic,Passaic,84136.0
4,Passaic,NJ,Passaic,Passaic,69781.0
5,Union City,NJ,Hudson,Hudson,66455.0
6,East Orange,NJ,Essex,Essex,64270.0
7,North Bergen,NJ,Hudson,Hudson,60773.0
8,Wayne,NJ,Passaic,Passaic,54717.0
9,Hoboken,NJ,Hudson,Hudson,50005.0


Thank you to https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/ for assistance in merging and joining dataframes.

---

## Join highpop dataframes together

Create a new dataframe that joins all 3 state highpop dataframes together.

In [43]:
# join 3 state highpop dfs together to create a new df
df_new = pd.concat([df_ny_highpop, df_ct_highpop, df_nj_highpop])
df_new

,City,State,County_x,County_y,Pop2010
0,Hempstead,NY,Nassau,Nassau,759757.0
1,Oyster Bay,NY,Nassau,Nassau,293214.0
2,Huntington,NY,Suffolk,Suffolk,203264.0
3,Smithtown,NY,Suffolk,Suffolk,117801.0
4,Rye,NY,Westchester,Westchester,45928.0
5,Monroe,NY,Orange,Orange,39912.0
6,Ossining,NY,Westchester,Westchester,37674.0
7,Haverstraw,NY,Rockland,Rockland,36634.0
8,Carmel,NY,Putnam,Putnam,34305.0
9,Eastchester,NY,Westchester,Westchester,32363.0


In [44]:
# reset index
df_new = df_new.reset_index(drop=True) # drops the previous index column
df_new.shape

(60, 5)

The new dataframe has 60 cities.  

For the sake of this project, "cities" are deemed any area with a population of 20000 or greater. The above 60 cities meet this definition, but not all of the cities from the previous dataframes did, hence they were filtered by population size.

---

## Finish preparing for retrieval of coordinates

In [45]:
# create a new column called City, State (makes it easier to find coordinates)
df_new['City, State'] = df_new[['City', 'State']].apply(', '.join, axis=1)

# remove original City and State columns
columns = ['City', 'State']
df_new.drop(columns, axis=1, inplace=True)
df_new.head()

,County_x,County_y,Pop2010,"City, State"
0,Nassau,Nassau,759757.0,"Hempstead, NY"
1,Nassau,Nassau,293214.0,"Oyster Bay, NY"
2,Suffolk,Suffolk,203264.0,"Huntington, NY"
3,Suffolk,Suffolk,117801.0,"Smithtown, NY"
4,Westchester,Westchester,45928.0,"Rye, NY"


In [46]:
# sort the City, State column in descending order
df_new = df_new.sort_values(by='City, State', ascending=True)

# reset index
df_new = df_new.reset_index(drop=True) # drops the previous index column
df_new.head()

,County_x,County_y,Pop2010,"City, State"
0,Westchester,Westchester,4330.0,"Armonk, NY"
1,Essex,Essex,35926.0,"Belleville, NJ"
2,Bergen,Bergen,26652.0,"Bergenfield, NJ"
3,Essex,Essex,47315.0,"Bloomfield, NJ"
4,Fairfield,Fairfield County,144229.0,"Bridgeport, CT"


---

## Find coordinates of each city

In [47]:
#install geocoder package for retrieving lat and long coordinates
!pip install geocoder
print('... geocoder installed!')

... geocoder installed!


In [48]:
import geocoder

In [49]:
latitude = [] # brackets represent a blank list (will put result into these)
longitude = []

# create a for loop to retrieve coordinates for each City, State
for place in df_new['City, State']:
    g = geocoder.arcgis('{}'.format(place))
    lat_lng_coords = g.latlng
    print(place, lat_lng_coords) # commenting out so won't see long wall of code on GitHub - uncomment to ensure correct results
    
    latitude.append(lat_lng_coords[0]) # use append and [0] to add first index of lat_lng_coords to the latitude variable
    longitude.append(lat_lng_coords[1])
    
## wait about a minute for the loop to cycle through
## potential error: connection may time out, if so, run cell again and wait

Armonk, NY [41.12188000000003, -73.70869999999996]
Belleville, NJ [40.79546000000005, -74.14924999999994]
Bergenfield, NJ [40.93480000000005, -73.99546999999995]
Bloomfield, NJ [40.79191000000003, -74.19574999999998]
Bridgeport, CT [41.18213000000003, -73.19063999999997]
Carmel, NY [41.42798000000005, -73.67673999999994]
Cliffside Park, NJ [40.82125000000008, -73.98799999999994]
Clifton, NJ [40.86575000000005, -74.15981999999997]
Danbury, CT [41.39268000000004, -73.45358999999996]
Darien, CT [41.06908000000004, -73.47752999999994]
East Orange, NJ [40.76099000000005, -74.20990999999998]
Eastchester, NY [40.95633000000004, -73.81517999999994]
Englewood, NJ [40.89525000000003, -73.97459999999995]
Fair Lawn, NJ [40.93610000000007, -74.13190999999995]
Fairfield, CT [41.16633000000007, -73.22875999999997]
Fishkill, NY [41.535330000000044, -73.90290999999996]
Fort Lee, NJ [40.85339000000005, -73.97427999999996]
Garfield, NJ [40.88137000000006, -74.11343999999997]
Greenwich, CT [41.06425000000

In [50]:
# add a Latitude and Longitude column to the dataframe - populate with latitude and longitude lists
df_new['Latitude'] = latitude
df_new['Longitude'] = longitude
df_new.head()

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
0,Westchester,Westchester,4330.0,"Armonk, NY",41.12188,-73.70870
1,Essex,Essex,35926.0,"Belleville, NJ",40.79546,-74.14925
2,Bergen,Bergen,26652.0,"Bergenfield, NJ",40.93480,-73.99547
3,Essex,Essex,47315.0,"Bloomfield, NJ",40.79191,-74.19575
4,Fairfield,Fairfield County,144229.0,"Bridgeport, CT",41.18213,-73.19064


#### Examine City, State if few nearby venues appear (Foursquare section)

Investigate cities with 10 or less nearby venues:
* Armonk, NY       (6 venues) - leaving as is because coordinates specified for IBM's HQ
* Carmel, NY       (7 venues) - 
* Darien, CT       (7 venues) - 
* East Orange, NJ  (8 venues) - 
* Greenwich, CT    (0 venues) -
* New Windsor, NY  (4 venues) - 
* North Bergen, NJ (8 venues) - 
* Norwalk, CT      (7 venues) - 
* Paramus, NJ      (2 venues) - 
* Shelton, CT      (4 venues) - 
* Smithtown, NY    (5 venues) - 
* Trumbull, CT     (2 venues) - 

In [51]:
# examine the row with Greenwich, CT
df_new[df_new['City, State'].str.contains("Greenwich")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
18,Fairfield,Fairfield County,61171.0,"Greenwich, CT",41.06425,-73.63977


In [52]:
# altering Greenwich, CT's coordinates so more accurately placed atop the city center
# if view the map below *before* altering coordinates, the rest of the nearby city markers are closer to the coast
# while Greenwich's was placed more inland, despite having a city close to the coast as well
# did a search via GoogleMaps and right-clicked the location of the city name to determine coordinates
df_new.loc[18, 'Latitude'] = 41.02631
df_new.loc[18, 'Longitude'] = -73.62811

In [53]:
# verify changes made
df_new[df_new['City, State'].str.contains("Greenwich")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
18,Fairfield,Fairfield County,61171.0,"Greenwich, CT",41.02631,-73.62811


In [54]:
# examine the row with Darien, CT
df_new[df_new['City, State'].str.contains("Darien")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
9,Fairfield,Fairfield County,20732.0,"Darien, CT",41.06908,-73.47753


In [55]:
df_new.loc[9, 'Latitude'] = 41.07723
df_new.loc[9, 'Longitude'] = -73.46866

In [56]:
# examine the row with East Orange, NJ
df_new[df_new['City, State'].str.contains("East Orange")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
10,Essex,Essex,64270.0,"East Orange, NJ",40.76099,-74.20991


In [57]:
df_new.loc[10, 'Latitude'] = 40.76708
df_new.loc[10, 'Longitude'] = -74.20511

In [58]:
# examine the row with North Bergen, NJ
df_new[df_new['City, State'].str.contains("North Bergen")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
37,Hudson,Hudson,60773.0,"North Bergen, NJ",40.79301,-74.02038


In [59]:
df_new.loc[37, 'Latitude'] = 40.80425
df_new.loc[37, 'Longitude'] = -74.01229

In [60]:
# examine the row with Norwalk, CT
df_new[df_new['City, State'].str.contains("Norwalk")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
38,Fairfield,Fairfield County,85603.0,"Norwalk, CT",41.11366,-73.40765


In [61]:
df_new.loc[38, 'Latitude'] = 41.11769
df_new.loc[38, 'Longitude'] = -73.40826

In [62]:
# examine the row with Paramus, NJ
df_new[df_new['City, State'].str.contains("Paramus")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
42,Bergen,Bergen,26164.0,"Paramus, NJ",40.92712,-74.06176


In [63]:
df_new.loc[42, 'Latitude'] = 40.94459
df_new.loc[42, 'Longitude'] = -74.07559

In [64]:
# examine the row with Shelton, CT
df_new[df_new['City, State'].str.contains("Shelton")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
48,Fairfield,Fairfield County,39559.0,"Shelton, CT",41.3226,-73.1091


In [65]:
df_new.loc[48, 'Latitude'] = 41.31653
df_new.loc[48, 'Longitude'] = -73.09305

In [66]:
# examine the row with Smithtown, NY
df_new[df_new['City, State'].str.contains("Smithtown")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
49,Suffolk,Suffolk,117801.0,"Smithtown, NY",40.84723,-73.23461


In [67]:
df_new.loc[49, 'Latitude'] = 40.85593
df_new.loc[49, 'Longitude'] = -73.20054

In [68]:
# examine the row with Trumbull, CT
df_new[df_new['City, State'].str.contains("Trumbull")]

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude
54,Fairfield,Fairfield County,36018.0,"Trumbull, CT",41.23495,-73.21967


In [69]:
df_new.loc[54, 'Latitude'] = 41.24298
df_new.loc[54, 'Longitude'] = -73.20049

---

## Map city coordinates

In [70]:
## NOTE: might want to change Armonk's coordinates to IBM headquarters coordinates instead.
### refer to Intro & Data notepad (has HQ coordinates)

In [71]:
# find Armonk's latitude and longitude coordinates
g = geocoder.arcgis('Armonk, New York')
g.latlng

[41.12188000000003, -73.70869999999996]

In [72]:
import folium # map rendering library

# create a map of Armonk
map_armonk = folium.Map(location=[41.12188, -73.70870], zoom_start=9)

# add city, state markers
for lat, lng, place in zip(df_new['Latitude'], df_new['Longitude'], df_new['City, State']):
    label = '{}'.format(place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_armonk)  
    
map_armonk

---

## Use Foursquare to retrieve nearby venue data, and put into a *nearby_venues* dataframe

In [73]:
# write down your Foursquare credentials and version
CLIENT_ID = 'OTTSMWX5Y1XYDLR3OWHUWVEVDTH1HO3N2BKCTJD4JMGNZYIH' # your Foursquare ID
CLIENT_SECRET = 'TTST1GFSD4HKPE4VQAWJB3YVLDGZ2VIZTBI03BZQTXFI5FYA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: OTTSMWX5Y1XYDLR3OWHUWVEVDTH1HO3N2BKCTJD4JMGNZYIH
CLIENT_SECRET:TTST1GFSD4HKPE4VQAWJB3YVLDGZ2VIZTBI03BZQTXFI5FYA


In [74]:
LIMIT = 100 # limits number of returned venues

## NOTE: might want to change limit size
## might want to change radius size below too

In [75]:
pd.set_option('display.max_rows', None)

Create the user-defined function getNearbyVenues.

In [76]:
# will return up to 100 venues within 500m of each city
def getNearbyVenues(names, latitudes, longitudes, radius=805): #805m = ~half a mile
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [77]:
# create a new dataframe of venues near cities near IBM's headquarters
## (will print only city names when applying the function)
armonk_venues = getNearbyVenues(names = df_new['City, State'],
                                   latitudes = df_new['Latitude'],
                                   longitudes = df_new['Longitude']
                                  )

# wait about 30 seconds
# if an error appears about 'groups', re-run the cell

Armonk, NY
Belleville, NJ
Bergenfield, NJ
Bloomfield, NJ
Bridgeport, CT
Carmel, NY
Cliffside Park, NJ
Clifton, NJ
Danbury, CT
Darien, CT
East Orange, NJ
Eastchester, NY
Englewood, NJ
Fair Lawn, NJ
Fairfield, CT
Fishkill, NY
Fort Lee, NJ
Garfield, NJ
Greenwich, CT
Hackensack, NJ
Harrison, NY
Haverstraw, NY
Hempstead, NY
Hoboken, NJ
Huntington, NY
Jersey City, NJ
Kearny, NJ
Lodi, NJ
Lyndhurst, NJ
Mahwah, NJ
Mamaroneck, NY
Monroe, NY
Montclair, NJ
New Windsor, NY
Newark, NJ
Newburgh, NY
Newtown, CT
North Bergen, NJ
Norwalk, CT
Nutley, NJ
Ossining, NY
Oyster Bay, NY
Paramus, NJ
Passaic, NJ
Paterson, NJ
Ridgefield, CT
Ridgewood, NJ
Rye, NY
Shelton, CT
Smithtown, NY
Somers, NY
Stamford, CT
Stratford, CT
Teaneck, NJ
Trumbull, CT
Union City, NJ
Wayne, NJ
West Milford, NJ
West New York, NJ
Westport, CT


In [78]:
armonk_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Armonk, NY",41.12188,-73.7087,Equinox Armonk,41.120953,-73.708910,Gym
1,"Armonk, NY",41.12188,-73.7087,DeCicco & Sons,41.126228,-73.712832,Supermarket
2,"Armonk, NY",41.12188,-73.7087,Fortina,41.126450,-73.712623,Italian Restaurant
3,"Armonk, NY",41.12188,-73.7087,DeCicco's Pub,41.126124,-73.712678,Pub
4,"Armonk, NY",41.12188,-73.7087,Amore Pizzeria & Pasta,41.124916,-73.714917,Italian Restaurant


In [79]:
print("There are {} venues close to cities within a 35mi commute to IBM's headquarters.".format(armonk_venues.shape[0]))

There are 2848 venues close to cities within a 35mi commute to IBM's headquarters.


In [80]:
# see the distribution of how many venues are near each city
armonk_venues.groupby('City')['Venue'].count()

City
Armonk, NY             32
Belleville, NJ         44
Bergenfield, NJ        47
Bloomfield, NJ         37
Bridgeport, CT         48
Carmel, NY             18
Cliffside Park, NJ     54
Clifton, NJ            22
Danbury, CT            45
Darien, CT             61
East Orange, NJ        13
Eastchester, NY        46
Englewood, NJ          59
Fair Lawn, NJ          33
Fairfield, CT          35
Fishkill, NY           42
Fort Lee, NJ          100
Garfield, NJ           30
Greenwich, CT          87
Hackensack, NJ         56
Harrison, NY           37
Haverstraw, NY         16
Hempstead, NY          44
Hoboken, NJ           100
Huntington, NY         94
Jersey City, NJ       100
Kearny, NJ             40
Lodi, NJ               36
Lyndhurst, NJ          55
Mahwah, NJ             14
Mamaroneck, NY         67
Monroe, NY             27
Montclair, NJ         100
New Windsor, NY         6
Newark, NJ             81
Newburgh, NY           37
Newtown, CT            29
North Bergen, NJ       58
Norwalk

In [81]:
# determine the number of different/unique venue categories
print('There are {} unique venue categories.'.format(len(armonk_venues['Venue Category'].unique())))

There are 283 unique venue categories.


---

## Create a *armonk_onehot* dataframe to show the distribution of venue categories

In [82]:
pd.set_option('display.max_columns', None) # so can see all of the columns (otherwise will see ... in the middle)

## create a new dataframe of dummy variables to see the venue category of venue near each city
# perform one hot encoding
armonk_onehot = pd.get_dummies(armonk_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
armonk_onehot['City'] = armonk_venues['City']

# move city column to the beginning
col_at_beg = ['City']
armonk_onehot = armonk_onehot[[i for i in col_at_beg] 
        + [i for i in armonk_onehot if i not in col_at_beg]] 

armonk_onehot.head(10)

,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game St

In [83]:
armonk_onehot.shape

(2848, 283)

---

## Group by city and visualize the mean frequency per venue category

In [84]:
# use groupby to combine venues' data within the same city into one row each
# and show the mean frequency of venue category of venues within each city
armonk_grouped = armonk_onehot.groupby('City').mean().reset_index()
armonk_grouped

,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game St

In [85]:
armonk_grouped.shape

(60, 283)

---

## Determine most-frequent venue categories per city and put this into a dataframe

In [86]:
# determine the 5 most-frequent venue category types nearby each city
num_top_venues = 5

for place in armonk_grouped['City']:
    print("----"+place+"----")
    
    temp = armonk_grouped[armonk_grouped['City'] == place].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n') # adds a blank line separating each city temp table

----Armonk, NY----
                     venue  freq
0                     Park  0.06
1      American Restaurant  0.06
2       Italian Restaurant  0.06
3        Recreation Center  0.03
4  New American Restaurant  0.03


----Belleville, NJ----
                venue  freq
0         Pizza Place  0.14
1  Italian Restaurant  0.07
2   Martial Arts Dojo  0.05
3            Pharmacy  0.05
4   Convenience Store  0.05


----Bergenfield, NJ----
                venue  freq
0         Pizza Place  0.09
1  Italian Restaurant  0.06
2        Dance Studio  0.06
3      Sandwich Place  0.06
4                Bank  0.04


----Bloomfield, NJ----
         venue  freq
0  Gas Station  0.05
1         Bank  0.05
2        Diner  0.05
3   Donut Shop  0.05
4   Steakhouse  0.03


----Bridgeport, CT----
                venue  freq
0           Juice Bar  0.06
1          Donut Shop  0.06
2  Chinese Restaurant  0.04
3              Lawyer  0.04
4                Café  0.04


----Carmel, NY----
                venue  freq
0  

In [87]:
# create a user-defined function called return_most_common_venues
# will sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
pd.set_option('display.max_rows', None)

In [89]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd'] # word part of 1st, 2nd, 3rd
columns = ['City']

# create columns according to number of top venues
# for loop where column number increases by one and suffixes added as st, nd, rd then only with th
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) 
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe with the 10 most frequent nearby venues by category for each city
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = armonk_grouped['City']

for ind in np.arange(armonk_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(armonk_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Armonk, NY",American Restaurant,Park,Italian Restaurant,Indian Restaurant,Mexican Restaurant,Supermarket,Gastropub,Gas Station,Sporting Goods Shop,Spa
1,"Belleville, NJ",Pizza Place,Italian Restaurant,Sandwich Place,Martial Arts Dojo,Gym,Chinese Restaurant,Pharmacy,Convenience Store,Shopping Plaza,Clothing Store
2,"Bergenfield, NJ",Pizza Place,Italian Restaurant,Dance Studio,Sandwich Place,Bagel Shop,Fast Food Restaurant,Bar,BBQ Joint,Convenience Store,Pharmacy
3,"Bloomfield, NJ",Diner,Bank,Gas Station,Donut Shop,Ice Cream Shop,Shoe Store,Market,Steakhouse,Martial Arts Dojo,Southern / Soul Food Restaurant
4,"Bridgeport, CT",Donut Shop,Juice Bar,Lawyer,Italian Restaurant,Chinese Restaurant,Café,Spanish Restaurant,Concert Hall,Plaza,Latin American Restaurant
5,"Carmel, NY",Italian Restaurant,Diner,Lake,Mexican Restaurant,Tea Room,Sandwich Place,Thai Restaurant,Bank,Bagel Shop,Trail
6,"Cliffside Park, NJ",Turkish Restaurant,Spa,Mediterranean Restaurant,Grocery Store,Pizza Place,Chinese Restaurant,Sushi Restaurant,Café,Middle Eastern Restaurant,Mexican Restaurant
7,"Clifton, NJ",Pizza Place,Liquor Store,Pharmacy,Bank,Deli / Bodega,Rental Car Location,Flower Shop,Shopping Mall,Gym / Fitness Center,Bar
8,"Danbury, CT",Pizza Place,Bakery,Latin American Restaurant,Brazilian Restaurant,Gym / Fitness Center,Bank,Grocery Store,Mexican Restaurant,Pharmacy,Check Cashing Service
9,"Darien, CT",Pizza Place,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Sandwich Place,Burger Joint,Coffee Shop,Sporting Goods Shop,Donut Shop,Gas Station


---

## Use *k*-Means to create 5 city clusters, then visualize them on a map

In [90]:
## practice!
from scipy.cluster.hierarchy import dendrogram, linkage
X = armonk_grouped.drop('City', 1)

# generate the linkage matrix
Z = linkage(X, 'centroid')
Z

array([[ 24.        ,  30.        ,   0.13824801,   2.        ],
       [ 51.        ,  60.        ,   0.12669762,   3.        ],
       [ 32.        ,  61.        ,   0.12695544,   4.        ],
       [ 25.        ,  62.        ,   0.12333644,   5.        ],
       [  9.        ,  63.        ,   0.12270655,   6.        ],
       [ 18.        ,  64.        ,   0.12809214,   7.        ],
       [ 46.        ,  65.        ,   0.13038627,   8.        ],
       [ 39.        ,  66.        ,   0.13476351,   9.        ],
       [ 23.        ,  67.        ,   0.13827676,  10.        ],
       [ 12.        ,  68.        ,   0.13827023,  11.        ],
       [ 19.        ,  69.        ,   0.13738897,  12.        ],
       [ 28.        ,  70.        ,   0.13797437,  13.        ],
       [  0.        ,  71.        ,   0.14188262,  14.        ],
       [ 45.        ,  72.        ,   0.14294441,  15.        ],
       [ 47.        ,  73.        ,   0.1419305 ,  16.        ],
       [ 55.        ,  74

In [91]:
## practice!
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(X))
c

# want this number to be close to 1!
## use diff methods for linkage matrix to achieve this

0.9465253173190168

In [92]:
#!pip install yellowbrick

In [93]:
#from sklearn.cluster import KMeans
#from yellowbrick.cluster import KElbowVisualizer

# Instantiate the clustering model and visualizer
#model = KMeans()
#visualizer = KElbowVisualizer(model, k=(3,16))

#visualizer.fit(X)        # Fit the data to the visualizer
#visualizer.show()        # Finalize and render the figure

In [94]:
import matplotlib.pyplot as plt

#practice!!
last = Z[-10:, 2]
last_rev = last[::-1]
idxs = np.arange(1, len(last) + 1)
plt.plot(idxs, last_rev)

acceleration = np.diff(last, 2)  # 2nd derivative of the distances
acceleration_rev = acceleration[::-1]
plt.plot(idxs[:-2] + 1, acceleration_rev)
plt.show()
k = acceleration_rev.argmax() + 2  # if idx 0 is the max of this we want 2 clusters
print ("clusters:", k)

# determine optimal number of clusters

<Figure size 640x480 with 1 Axes>

clusters: 2


In [95]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit_transform(X)

In [106]:
from sklearn.cluster import KMeans # import k-means

kclusters = 2 # set number of clusters

# create a new dataframe and drop the City column
armonk_grouped_clustering = armonk_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(X) # k-means++ intelligently  
                                                                                                         # selects starter cluster centers

# check the cluster labels/assignments for each row
kmeans.labels_[0:59] 

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [107]:
# add clustering labels to city_venues_sorted dataframe
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# create a new dataframe containing the same data as df_new
armonk_merged = df_new

# merge the df_new (now armonk_merged - see above line) and city_venues_sorted dataframes into one - armonk_merged dataframe
# do so to have latitude and longitude values again
armonk_merged = armonk_merged.join(city_venues_sorted.set_index('City'), on='City, State')

armonk_merged

,County_x,County_y,Pop2010,"City, State",Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Westchester,Westchester,4330.0,"Armonk, NY",41.12188,-73.70870,0,American Restaurant,Park,Italian Restaurant,Indian Restaurant,Mexican Restaurant,Supermarket,Gastropub,Gas Station,Sporting Goods Shop,Spa
1,Essex,Essex,35926.0,"Belleville, NJ",40.79546,-74.14925,0,Pizza Place,Italian Restaurant,Sandwich Place,Martial Arts Dojo,Gym,Chinese Restaurant,Pharmacy,Convenience Store,Shopping Plaza,Clothing Store
2,Bergen,Bergen,26652.0,"Bergenfield, NJ",40.93480,-73.99547,0,Pizza Place,Italian Restaurant,Dance Studio,Sandwich Place,Bagel Shop,Fast Food Restaurant,Bar,BBQ Joint,Convenience Store,Pharmacy
3,Essex,Essex,47315.0,"Bloomfield, NJ",40.79191,-74.19575,0,Diner,Bank,Gas Station,Donut Shop,Ice Cream Shop,Shoe Store,Market,Steakhouse,Martial Arts Dojo,Southern / Soul Food Restaurant
4,Fairfield,Fairfield County,144229.0,"Bridgeport, CT",41.18213,-73.19064,0,Donut Shop,Juice Bar,Lawyer,Italian Restaurant,Chinese Restaurant,Café,Spanish Restaurant,Concert Hall,Plaza,Latin American Restaurant
5,Putnam,Putnam,34305.0,"Carmel, NY",41.42798,-73.67674,0,Italian Restaurant,Diner,Lake,Mexican Restaurant,Tea Room,Sandwich Place,Thai Restaurant,Bank,Bagel Shop,Trail
6,Bergen,Bergen,23510.0,"Cliffside Park, NJ",40.82125,-73.98800,1,Turkish Restaurant,Spa,Mediterranean Restaurant,Grocery Store,Pizza Place,Chinese Restaurant,Sushi Restaurant,Café,Middle Eastern Restaurant,Mexican Restaurant
7,Passaic,Passaic,84136.0,"Clifton, NJ",40.86575,-74.15982,0,Pizza Place,Liquor Store,Pharmacy,Bank,Deli / Bodega,Rental Car Location,Flower Shop,Shopping Mall,Gym / Fitness Center,Bar
8,Fairfield,Fairfield County,80893.0,"Danbury, CT",41.39268,-73.45359,1,Pizza Place,Bakery,Latin American Restaurant,Brazilian Restaurant,Gym / Fitness Center,Bank,Grocery Store,Mexican Restaurant,Pharmacy,Check Cashing Service
9,Fairfield,Fairfield County,20732.0,"Darien, CT",41.07723,-73.46866,1,Pizza Place,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Sandwich Place,Burger Joint,Coffee Shop,Sporting Goods Shop,Donut Shop,Gas Station


In [108]:
city_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)

In [109]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[41.12188, -73.70870], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(armonk_merged['Latitude'], armonk_merged['Longitude'], armonk_merged['City, State'], armonk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---

## Examine each cluster more in depth

#### Cluster 0

In [110]:
armonk_merged.loc[armonk_merged['Cluster Labels'] == 0, armonk_merged.columns[[3] + list(range(6, armonk_merged.shape[1]))]]

,"City, State",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Armonk, NY",0,American Restaurant,Park,Italian Restaurant,Indian Restaurant,Mexican Restaurant,Supermarket,Gastropub,Gas Station,Sporting Goods Shop,Spa
1,"Belleville, NJ",0,Pizza Place,Italian Restaurant,Sandwich Place,Martial Arts Dojo,Gym,Chinese Restaurant,Pharmacy,Convenience Store,Shopping Plaza,Clothing Store
2,"Bergenfield, NJ",0,Pizza Place,Italian Restaurant,Dance Studio,Sandwich Place,Bagel Shop,Fast Food Restaurant,Bar,BBQ Joint,Convenience Store,Pharmacy
3,"Bloomfield, NJ",0,Diner,Bank,Gas Station,Donut Shop,Ice Cream Shop,Shoe Store,Market,Steakhouse,Martial Arts Dojo,Southern / Soul Food Restaurant
4,"Bridgeport, CT",0,Donut Shop,Juice Bar,Lawyer,Italian Restaurant,Chinese Restaurant,Café,Spanish Restaurant,Concert Hall,Plaza,Latin American Restaurant
5,"Carmel, NY",0,Italian Restaurant,Diner,Lake,Mexican Restaurant,Tea Room,Sandwich Place,Thai Restaurant,Bank,Bagel Shop,Trail
7,"Clifton, NJ",0,Pizza Place,Liquor Store,Pharmacy,Bank,Deli / Bodega,Rental Car Location,Flower Shop,Shopping Mall,Gym / Fitness Center,Bar
10,"East Orange, NJ",0,American Restaurant,Donut Shop,Discount Store,Chinese Restaurant,Liquor Store,Bus Stop,Restaurant,Caribbean Restaurant,Pizza Place,Food
11,"Eastchester, NY",0,Pharmacy,Pizza Place,Italian Restaurant,Pub,Bar,Hardware Store,Martial Arts Dojo,Chinese Restaurant,Sandwich Place,American Restaurant
12,"Englewood, NJ",0,Mexican Restaurant,Pizza Place,Japanese Restaurant,Bank,Gym,Italian Restaurant,Thai Restaurant,Middle Eastern Restaurant,Deli / Bodega,Optical Shop


Cluster 0 cities have various restaurants and pizza places nearby - a good fit for those who frequently eat out.

#### Cluster 1

In [111]:
armonk_merged.loc[armonk_merged['Cluster Labels'] == 1, armonk_merged.columns[[3] + list(range(6, armonk_merged.shape[1]))]]

,"City, State",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Cliffside Park, NJ",1,Turkish Restaurant,Spa,Mediterranean Restaurant,Grocery Store,Pizza Place,Chinese Restaurant,Sushi Restaurant,Café,Middle Eastern Restaurant,Mexican Restaurant
8,"Danbury, CT",1,Pizza Place,Bakery,Latin American Restaurant,Brazilian Restaurant,Gym / Fitness Center,Bank,Grocery Store,Mexican Restaurant,Pharmacy,Check Cashing Service
9,"Darien, CT",1,Pizza Place,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Sandwich Place,Burger Joint,Coffee Shop,Sporting Goods Shop,Donut Shop,Gas Station
16,"Fort Lee, NJ",1,Korean Restaurant,Asian Restaurant,Japanese Restaurant,Bakery,Café,Bank,Sandwich Place,Rental Car Location,Mobile Phone Shop,Spa
18,"Greenwich, CT",1,Italian Restaurant,Clothing Store,Bank,Coffee Shop,Women's Store,Gym / Fitness Center,French Restaurant,Sushi Restaurant,Café,Supermarket
23,"Hoboken, NJ",1,Bakery,Gym / Fitness Center,Park,Coffee Shop,American Restaurant,Pizza Place,Cuban Restaurant,Pub,Sushi Restaurant,Smoke Shop
24,"Huntington, NY",1,Bar,Italian Restaurant,Pizza Place,Mexican Restaurant,Ice Cream Shop,Bank,Bakery,Deli / Bodega,Grocery Store,New American Restaurant
25,"Jersey City, NJ",1,American Restaurant,Coffee Shop,Café,Pizza Place,Taco Place,Restaurant,Gym / Fitness Center,Grocery Store,Beer Garden,Japanese Restaurant
30,"Mamaroneck, NY",1,Italian Restaurant,American Restaurant,Mexican Restaurant,Pizza Place,Bank,Donut Shop,Asian Restaurant,Baseball Field,Bakery,BBQ Joint
32,"Montclair, NJ",1,Café,Italian Restaurant,Bakery,Spa,Gym,Yoga Studio,Ice Cream Shop,Juice Bar,Pizza Place,Seafood Restaurant


Cluster 1 cities also have various restaurants and pizza places nearby - a good fit for those who like to eat out.